# NYC Buildings

<img align='right' style='margin:8px' src='./assets/dashboard.png'/>

Many plotting libraries can handle collections of polygons, including [Bokeh](https://docs.bokeh.org/en/latest/docs/examples/topics/geo/texas_hover_map.html) and [HoloViews](https://holoviews.org/gallery/demos/bokeh/choropleth_data_link.html). However, because browser-based libraries like Bokeh and Plotly send all the polygon data to Javascript running in the browser, they can struggle when either the collections or the individual polygons themselves get large. Even natively in Python, typical formats like Shapely for representing polygons scale poorly to large polygon collections, because each polygon is wrapped up as a full, separate Python object, leading to a lot of duplicated storage overhead when many polygons of the same type are defined.

If you want to work with lots of polygons, here you can see how to use [SpatialPandas](https://github.com/holoviz/spatialpandas) and Dask to represent polygons efficiently in memory, and [Datashader](https://datashader.org) to render them quickly in a web browser.

This example plots the outlines of all one million+ buildings in New York City. See
[nyc.gov](https://nyc.maps.arcgis.com/home/item.html?id=870bf69e8a8044aea4488e564c0b4010#overview) for the original data and its description.

In [ ]:
import hvplot.dask # noqa
import hvplot.pandas # noqa
import datashader as ds
import colorcet as cc
import spatialpandas as spd
import spatialpandas.io

from dask.diagnostics import ProgressBar
from holoviews import opts
from holoviews.streams import PlotSize

In [ ]:
# Add more resolution to dynamic plots, particularly important for Retina displays
PlotSize.scale=2.0
opts.defaults(
    opts.Polygons(width=600, height=500, xaxis=None, yaxis=None))

from dask.distributed import Client
client = Client()

client

In [ ]:
ddf = spd.io.read_parquet_dask('./data/nyc_buildings.parq').persist()
print(len(ddf))
ddf.head()

Here you can see that we have 1.1 million "MultiPolygons", some of which have a `type` and `name` declared.

To get a look at this data, let's plot all the polygons, overlaid on a tiled map of the region:

In [ ]:
ddf.hvplot.polygons(tiles='CartoLight', rasterize=True, aggregator='any')

At this scale, the plot looks like a bunch of dots or large colored areas, because each building is smaller than a pixel in the plot. But if you have a live Python server running, you can use the Bokeh tools to zoom in and have the plot dynamically redrawn, showing you the full outline of each polygon.

Alternatively, we can re-plot the data over a smaller selection of the location that can show us the indidvidual polygons using the `xlim` and `ylim` parameters:

In [ ]:
ddf.hvplot.polygons(tiles='CartoLight', rasterize=True, aggregator='any',
                    xlim=(-8231400, -8230900), ylim=(4971900, 4972600))

Now let's make use of the category information. To get a manageable number of types, we'll compute the top 10 most common categories and drop everything else:

In [ ]:
cats = list(ddf['type'].value_counts().compute().iloc[:10].index.values) + ['unknown']

ddf['type'] = ddf['type'].replace({None: 'unknown'})
ddf = ddf[ddf['type'].isin(cats)]
ddf['type'] = ddf['type'].astype('category').cat.as_known()

SpatialPandas lets us build a spatial index for accessing spatially organized regions more quickly, so let's do that:

In [ ]:
with ProgressBar():
    ddf = ddf.build_sindex().persist()

Now we can view each category separately with a selector widget:

In [ ]:
ddf.hvplot.polygons(rasterize=True, tiles='CartoLight', groupby='type', aggregator='any')

If you look at each one, you can see that unfortunately most of the categories are unknown, but there are interesting patterns (e.g. almost no garages in Manhattan, and apparently all the sheds are in New Jersey).

Since these buildings don't normally overlap, we can actually combine them all into a single plot using color to show all of the categories (though we have to construct a color key manually):

In [ ]:
colors    = cc.glasbey_bw_minc_20_maxl_70
color_key = {cat: tuple(int(e*255.) for e in colors[i]) for i, cat in enumerate(cats)}

Now we put it all together, showing the color-coded plot:

In [ ]:
plot = ddf.hvplot.polygons(tiles='CartoLight', data_aspect=1, datashade=True,
                           aggregator=ds.by('type', ds.any()), cmap=color_key)

plot

If you zoom into an area of interest, you'll see an overlay of the building at that location, with the color of the polygon telling you the type of building it is.

Finally, we'll make this notebook into a shareable app (run with `anaconda-project run dashboard`):

In [ ]:
import panel as pn
pn.extension()

text = """
# [1 million buildings in NYC](https://examples.holoviz.org/nyc_buildings)
## Rendered using [Datashader](https://datashader.org) and [hvPlot](https://hvplot.holoviz.org).
"""

pn.Column(text, pn.panel(plot, sizing_mode='stretch_both')).servable();